In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/Constitución Española/Constitución española 3-7.pdf")
documents = loader.load()
for document in documents:
    document.metadata['filename'] = document.metadata['source']

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(documents)

In [4]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

generator_llm = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_GPT3TURBO_DEPLOYMENT"],
)
critic_llm = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_GPT4TURBO_DEPLOYMENT"],
)
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_ADA2_DEPLOYMENT"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
)

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

/home/alvaro/TFM/code/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
testset = generator.generate_with_langchain_docs(documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

Generating: 100%|██████████| 20/20 [21:46<00:00, 65.31s/it]


In [6]:
df = testset.to_pandas()

In [7]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How is the right to choose one's residence and...,[Artículo 17. \n1. Toda persona tiene derecho...,The right to choose one's residence and to mov...,simple,[{'source': '../data/Constitución Española/Con...,True
1,How does the Spanish constitution typically en...,[Artículo 17. \n1. Toda persona tiene derecho...,The Spanish constitution ensures societal plur...,simple,[{'source': '../data/Constitución Española/Con...,True
2,What is the capital of Spain according to the ...,[TEXTO CONSOLIDADO\nÚltima modificación: 17 de...,nan,simple,[{'source': '../data/Constitución Española/Con...,True
3,What rights do Spanish citizens have in terms ...,[Artículo 17. \n1. Toda persona tiene derecho...,Spanish citizens have the right to choose thei...,simple,[{'source': '../data/Constitución Española/Con...,True
4,What are the current legal conditions for acqu...,[CAPÍTULO PRIMERO\nDe los españoles y los extr...,"The current legal conditions for acquiring, re...",simple,[{'source': '../data/Constitución Española/Con...,True


In [8]:
df.to_json('../data/Constitución Española/Constitución española 3-7_gpt4turbo_testset.json', orient='records')
